In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy
import pickle
from glob import glob
from typing import Any, Dict, List, Tuple, Union
import pandas as pd

"""Change to the data folder"""
new_path = "./new_train/new_train"
test_path = './new_val_in/new_val_in'
subset_test_path = './new_train/train_subset'
# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)
test_dataset = ArgoverseDataset(data_path=test_path)

### Create a loader to enable batch processing

In [3]:
batch_sz = 4

def train_agents_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = numpy.concatenate([numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch])
    out = numpy.concatenate([numpy.dstack([scene['p_out'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch])
    inp = torch.Tensor(inp)
    out = torch.Tensor(out)
    return [inp, out]

def train_all_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = numpy.concatenate([numpy.dstack([scene['p_in'][['dummy' not in word for word in scene['track_id'][:,0,0]],:,:]]) for scene in batch])
    out = numpy.concatenate([numpy.dstack([scene['p_out'][['dummy' not in word for word in scene['track_id'][:,0,0]],:,:]]) for scene in batch])
    inp = torch.Tensor(inp)
    out = torch.Tensor(out)
    return [inp, out]
    
def test_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = numpy.concatenate([numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch])
    inp = torch.Tensor(inp)
    idx = [numpy.dstack([scene['scene_idx']]) for scene in batch]
    return inp, idx
    
train_agent_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = True, collate_fn=train_agents_collate, num_workers=0)

train_all_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = True, collate_fn=train_all_collate, num_workers=0)

test_loader = DataLoader(test_dataset,batch_size=batch_sz, shuffle = True, collate_fn=test_collate, num_workers=0)

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class EncoderRNN(nn.Module):
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    def __init__(self,
                 input_size = 2,
                 embedding_size = 8,
                 hidden_size = 16):
        
        super(EncoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.linear = nn.Linear(input_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)

    def forward(self, x, hidden):
        embedded = F.relu(self.linear(x))
        hidden = self.lstm(embedded, hidden)
        return hidden


class DecoderRNN(nn.Module):
    """Decoder Network."""
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    def __init__(self, embedding_size=8, hidden_size=16, output_size=2):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.linear1 = nn.Linear(output_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)
        self.drop2 = nn.BatchNorm1d(embedding_size)

    def forward(self, x, hidden):
        embedded = F.relu(self.linear1(x))
        embedded = self.drop2(embedded)
        hidden = self.lstm(embedded, hidden)
        output = self.linear2(hidden[0])
        return output, hidden


In [5]:
from tqdm import tqdm_notebook as tqdm

def train(encoder, decoder, device, train_loader, encoder_optimizer, decoder_optimizer, epoch, log_interval=10000):    
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    iterator = tqdm(train_loader, total=int(len(train_loader)))
    counter = 0
    criterion = nn.MSELoss()
    
    for i_batch, sample_batch in enumerate(train_loader):
        
        inp, out = sample_batch
#         print(inp.shape)
        # preprocessing more ????
#         inp = inp[:,0,:,:]
#         out = out[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(inp.shape[0]):
            x_offset.append(inp[i][0][0].detach().clone())
            y_offset.append(inp[i][0][1].detach().clone())
    
        for j in range(inp.shape[0]):
            for i in range(inp.shape[1]):
                inp[j][i][0] = inp[j][i][0] - x_offset[j]
                inp[j][i][1] = inp[j][i][1] - y_offset[j]

        #outoput whatever
        for j in range(out.shape[0]):
            for i in range(out.shape[1]):
                out[j][i][0] = out[j][i][0] - x_offset[j]
                out[j][i][1] = out[j][i][1] - y_offset[j]
        
        _input, target = inp.to(device), out.to(device)
        
        encoder.train()
        decoder.train()
        
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        
        #encoder 
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        output_length = target.shape[1]
        feature_len = _input.shape[2]
        input_shape = _input.shape[2]
        
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
        
        loss = 0
        
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(target.shape).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Update loss
            loss += torch.sqrt(criterion(decoder_output[:, :2], target[:, di, :2]))

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        # Get average loss for pred_len
        loss = loss / 30

        # Backpropagate
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()
        
#         file1 = open("loss_steps.txt", "a")  # append mode
#         file1.write(str(loss.item()) + ",")
#         file1.close()
        
#       output = model(data)
#         loss = MSELoss(output, target)
        counter += 1
        iterator.set_postfix(loss=(loss.item()))
    

In [6]:
device = "cuda"
encoder = EncoderRNN(input_size=2)
decoder = DecoderRNN(output_size=2)

encoder = nn.DataParallel(encoder)
decoder = nn.DataParallel(decoder)

encoder.to(device)
decoder.to(device)

encoder_optimizer = torch.optim.Adam(encoder.parameters())
decoder_optimizer = torch.optim.Adam(decoder.parameters())

num_epoch = 6

for epoch in range(1, num_epoch + 1):
    train(encoder, decoder, device, train_all_loader, encoder_optimizer, decoder_optimizer, epoch)
#         train(encoder, decoder, device, train_agent_loader, encoder_optimizer, decoder_optimizer, epoch)
#         predict(model, device, test_loader)

In [7]:
def infer_absolute(
        test_loader: torch.utils.data.DataLoader,
        encoder: EncoderRNN,
        decoder: DecoderRNN,
#         start_idx: int,
#         forecasted_save_dir: str,
#         model_utils: ModelUtils,
):
    """Infer function for non-map LSTM baselines and save the forecasted trajectories.
    
    referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting
    
    Args:
        test_loader: DataLoader for the test set
        encoder: Encoder network instance
        decoder: Decoder network instance
        start_idx: start index for the current joblib batch
        forecasted_save_dir: Directory where forecasted trajectories are to be saved
        model_utils: ModelUtils instance

    """
    
    forecasted_trajectories = {}

    for i, (_input, idx) in enumerate(test_loader):
        
#         _input = _input[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(_input.shape[0]):
            x_offset.append(_input[i][0][0].detach().clone())
            y_offset.append(_input[i][0][1].detach().clone())
    
        for j in range(_input.shape[0]):
            for i in range(_input.shape[1]):
                _input[j][i][0] = _input[j][i][0] - x_offset[j]
                _input[j][i][1] = _input[j][i][1] - y_offset[j]

        _input = _input.to(device)

        # Set to eval mode
        encoder.eval()
        decoder.eval()

        # Encoder
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        input_shape = _input.shape[2]

        # Initialize encoder hidden state
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
       
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(
            (batch_size, 30, 2)).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        for i in range(30):
            for j in range(batch_size):
                decoder_outputs[j,i,0] = decoder_outputs[j,i,0] + x_offset[j]
                decoder_outputs[j,i,1] = decoder_outputs[j,i,1] + y_offset[j]
            
                if (idx[j][0][0][0] in forecasted_trajectories):
                    forecasted_trajectories[idx[j][0][0][0]].append(decoder_outputs[j,i,:].tolist())
                else:
                    forecasted_trajectories[idx[j][0][0][0]] = [decoder_outputs[j,i,:].tolist()]
                
    return(forecasted_trajectories)

In [8]:
output = infer_absolute(test_loader, encoder, decoder)

In [9]:
import pandas as pd
df = pd.DataFrame.from_dict(output, orient='index')
df

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
25942,"[887.3255004882812, 1266.4508056640625]","[887.795166015625, 1266.728271484375]","[888.2136840820312, 1266.9224853515625]","[888.6300659179688, 1267.1322021484375]","[889.0486450195312, 1267.3428955078125]","[889.4655151367188, 1267.55322265625]","[889.880859375, 1267.7635498046875]","[890.2941284179688, 1267.97412109375]","[890.7048950195312, 1268.1846923828125]","[891.11279296875, 1268.3953857421875]",...,"[895.2632446289062, 1270.627685546875]","[895.5980834960938, 1270.81298828125]","[895.92431640625, 1270.9940185546875]","[896.2418212890625, 1271.1705322265625]","[896.5504760742188, 1271.3424072265625]","[896.8501586914062, 1271.509765625]","[897.1409912109375, 1271.672607421875]","[897.4229125976562, 1271.8309326171875]","[897.6961059570312, 1271.98486328125]","[897.9605712890625, 1272.134765625]"
5590,"[1744.3702392578125, 456.7248229980469]","[1743.958740234375, 457.1307067871094]","[1743.687255859375, 457.4861145019531]","[1743.426025390625, 457.8392028808594]","[1743.1627197265625, 458.19647216796875]","[1742.8994140625, 458.55755615234375]","[1742.6353759765625, 458.92181396484375]","[1742.3704833984375, 459.2886962890625]","[1742.104248046875, 459.65753173828125]","[1741.83642578125, 460.0277099609375]",...,"[1738.6920166015625, 463.9759826660156]","[1738.3802490234375, 464.30426025390625]","[1738.06298828125, 464.62432861328125]","[1737.739990234375, 464.93560791015625]","[1737.4110107421875, 465.23748779296875]","[1737.0760498046875, 465.5294189453125]","[1736.7347412109375, 465.8108825683594]","[1736.3870849609375, 466.0813903808594]","[1736.032958984375, 466.34051513671875]","[1735.672119140625, 466.5877990722656]"
4945,"[1917.3382568359375, 506.5667724609375]","[1918.819091796875, 507.9087829589844]","[1920.1815185546875, 509.1021423339844]","[1921.5264892578125, 510.2807312011719]","[1922.8697509765625, 511.46551513671875]","[1924.2196044921875, 512.6693725585938]","[1925.578125, 513.8945922851562]","[1926.9427490234375, 515.1353149414062]","[1928.308349609375, 516.38134765625]","[1929.669189453125, 517.62158203125]",...,"[1943.9462890625, 529.9451904296875]","[1945.2589111328125, 531.0199584960938]","[1946.5955810546875, 532.097900390625]","[1947.956787109375, 533.168701171875]","[1949.338134765625, 534.2140502929688]","[1950.7288818359375, 535.209228515625]","[1952.1136474609375, 536.1292114257812]","[1953.474609375, 536.9574584960938]","[1954.79638671875, 537.6926879882812]","[1956.0675048828125, 538.3485107421875]"
32529,"[729.7630615234375, 1526.536376953125]","[729.786865234375, 1525.5762939453125]","[729.7674560546875, 1524.66064453125]","[729.7478637695312, 1523.7337646484375]","[729.7236938476562, 1522.810791015625]","[729.698486328125, 1521.90087890625]","[729.6710815429688, 1521.0010986328125]","[729.6417236328125, 1520.111572265625]","[729.6104125976562, 1519.23095703125]","[729.5770874023438, 1518.3572998046875]",...,"[729.1144409179688, 1508.783447265625]","[729.0726318359375, 1507.8990478515625]","[729.0333251953125, 1507.013427734375]","[728.9971313476562, 1506.1275634765625]","[728.9646606445312, 1505.2421875]","[728.9364013671875, 1504.358642578125]","[728.9129028320312, 1503.4775390625]","[728.8943481445312, 1502.599609375]","[728.8809204101562, 1501.7255859375]","[728.87255859375, 1500.855712890625]"
16565,"[737.6900024414062, 1531.012939453125]","[737.7388305664062, 1531.7119140625]","[737.7573852539062, 1532.2315673828125]","[737.7960205078125, 1532.7626953125]","[737.8322143554688, 1533.28759765625]","[737.8679809570312, 1533.8070068359375]","[737.9031372070312, 1534.321533203125]","[737.9376831054688, 1534.8310546875]","[737.971435546875, 1535.3355712890625]","[738.0042724609375, 1535.8350830078125]",...,"[738.2355346679688, 1540.9146728515625]","[738.2379760742188, 1541.3294677734375]","[738.2362670898438, 1541.735107421875]","[738.2302856445312, 1542.13134765625]","[738.2198486328125, 1542.5179443359375]","[738.20483398437

In [10]:
for i in range(30):
    df[['v{}'.format((i*2)+1), 'v{}'.format((i*2)+2)]] = pd.DataFrame(df.get(i).tolist(), index=df.index)

In [11]:
import numpy as np
dropped_cols = list(np.arange(30))
df2 = df.drop(dropped_cols, axis=1)
df2.index.name = 'ID'
df2

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60
ID,,,,,,,,,,,,,,,,,,,,,
25942,887.325500,1266.450806,887.795166,1266.728271,888.213684,1266.922485,888.630066,1267.132202,889.048645,1267.342896,...,896.850159,1271.509766,897.140991,1271.672607,897.422913,1271.830933,897.696106,1271.984863,897.960571,1272.134766
5590,1744.370239,456.724823,1743.958740,457.130707,1743.687256,457.486115,1743.426025,457.839203,1743.162720,458.196472,...,1737.076050,465.529419,1736.734741,465.810883,1736.387085,466.081390,1736.032959,466.340515,1735.672119,466.587799
4945,1917.338257,506.566772,1918.819092,507.908783,1920.181519,509.102142,1921.526489,510.280731,1922.869751,511.465515,...,1950.728882,535.209229,1952.113647,536.129211,1953.474609,536.957458,1954.796387,537.692688,1956.067505,538.348511
32529,729.763062,1526.536377,729.786865,1525.576294,729.767456,1524.660645,729.747864,1523.733765,729.723694,1522.810791,...,728.936401,1504.358643,728.912903,1503.477539,728.894348,1502.599609,728.880920,1501.725586,728.872559,1500.855713
16565,737.690002,1531.012939,737.738831,1531.711914,737.757385,1532.231567,737.796021,1532.762695,737.832214,1533.287598,...,738.204834,1542.895020,738.185120,1543.262207,738.160583,1543.619507,738.131165,1543.966919,738.096863,1544.304565
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27111,730.202881,1397.433838,730.264648,1396.515991,730.311829,1395.719360,730.346069,1394.901367,730.378784,1394.083130,...,730.978638,1377.635376,731.003113,1376.837646,731.028748,1376.040894,731.055786,1375.246216,731.084595,1374.454346
8112,729.371826,1517.247925,729.377502,1516.291016,729.347473,1515.407104,729.319214,1514.503662,729.283997,1513.597900,...,728.160034,1494.924194,728.126099,1494.045898,728.098572,1493.173462,728.077576,1492.307495,728.063232,1491.447998
16145,1980.554565,559.285461,1981.436768,560.169250,1982.282471,560.973877,1983.119629,561.780518,1983.959106,562.592712,...,2000.371460,578.989868,2001.018799,579.636658,2001.654907,580.272095,2002.280273,580.896790,2002.895874,581.511597


In [12]:
df2.to_csv("outputs6epb4all_batchnormDecoder.csv", index=True)